In [ ]:
# The purpose of this notbook is for writing python quickly and checking assumptions 
# without modifying existing notebooks

import keras
from keras.models import load_model
import cv2
import scipy
import numpy as np

In [ ]:
# model = load_model("test_model.h5")
model = load_model("attention_mask_model.h5")

In [ ]:
for i in range (-10, -1):
    print(i, "Layer: " + str(model.layers[i]))

In [ ]:
model.layers[-6]

In [ ]:
original_image = cv2.imread('./lesion_images/all_images_processed/' + "0001-0" + '.jpg')
X = cv2.resize(original_image, (224, 224))
X = np.true_divide(X, 255)
X = X.reshape(1, X.shape[0], X.shape[1], X.shape[2]) # reshape with batch size of 1

In [ ]:
print(original_image.shape)
print(X.shape)

In [ ]:
input_layer = model.layers[0].input
output_layer = model.layers[-1].output
output_weights = model.layers[-1].get_weights()[0]
last_conv_layer = model.layers[-6].output

print(input_layer.shape)
print(output_layer.shape)
print(last_conv_layer.shape)

In [ ]:
get_outputs = keras.backend.function([input_layer], [last_conv_layer, output_layer])

In [ ]:
[conv_outputs, predictions] = get_outputs([X])

In [ ]:
print(conv_outputs.shape)
print(conv_outputs[0,:,:,:].shape)
print(predictions.shape)

In [ ]:
first_conv_output = conv_outputs[0,:,:,:]
print(first_conv_output.shape)
mat_for_mult = scipy.ndimage.zoom(first_conv_output, (32, 32, 2.28571429), order=1) # dim: 224 x 224 x 1024
print(mat_for_mult.shape)

In [ ]:
print(output_weights[:, 0].shape)

In [ ]:
print(mat_for_mult.reshape(512 * 512, 512).shape)

In [ ]:
attention_map = np.dot(
        mat_for_mult.reshape(224 * 224, 512), 
        output_weights[:, 0]).reshape(224, 224)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.imshow(original_image)
ax.imshow(attention_map, cmap='jet', alpha=0.3)
plt.show()

In [ ]:
def generate_heatmap(model, image_id):
    '''
    Next Steps:
     - Integrate this into the original model.
     - Get a classification loss and an attention mining loss
    '''
    input_layer = model.layers[0].input
    output_layer = model.layers[-1].output
    output_weights = model.layers[-1].get_weights()[0]
    last_conv_layer = model.layers[-6].output

    get_outputs = keras.backend.function([input_layer], [last_conv_layer, output_layer])

    original_image = cv2.imread('./lesion_images/all_images_processed/' + image_id + '.jpg')
    X = cv2.resize(original_image, (512, 512))
    X = np.true_divide(X, 255)
    X = X.reshape(1, X.shape[0], X.shape[1], X.shape[2]) # reshape with batch size of 1

    [conv_outputs, predictions] = get_outputs([X])
    conv_outputs = conv_outputs[0, :, :, :]

    mat_for_mult = scipy.ndimage.zoom(conv_outputs, (32, 32, 0.5), order=1) # dim: 224 x 224 x 1024
    attention_map = np.dot(
        mat_for_mult.reshape(512 * 512, 512), 
        output_weights[:, 0]).reshape(512, 512)

    return attention_map